# 🚀 SelfAgent - 在 Colab 构建 Android APK

点击 **Runtime -> Run all** 一键构建

In [ ]:
# 1. 安装 Node.js
!curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
!sudo apt-get install -y nodejs
!node -v
print("✅ Node.js 安装完成")

In [ ]:
# 2. 安装 Java、Android SDK 和构建工具
!sudo apt-get install -y openjdk-17-jdk wget unzip

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

!mkdir -p ~/android-sdk/cmdline-tools
!wget -q https://dl.google.com/android/repository/commandlinetools-linux-9477386_latest.zip -O cmdline-tools.zip
!unzip -q -o cmdline-tools.zip -d ~/android-sdk/cmdline-tools
!mv ~/android-sdk/cmdline-tools/cmdline-tools ~/android-sdk/cmdline-tools/latest 2>/dev/null || true

os.environ['ANDROID_HOME'] = os.path.expanduser('~/android-sdk')
os.environ['ANDROID_SDK_ROOT'] = os.environ['ANDROID_HOME']
os.environ['PATH'] = os.environ['ANDROID_HOME'] + '/cmdline-tools/latest/bin:' + os.environ['ANDROID_HOME'] + '/platform-tools:' + os.environ['PATH']

!yes | ~/android-sdk/cmdline-tools/latest/bin/sdkmanager --licenses
!~/android-sdk/cmdline-tools/latest/bin/sdkmanager "platform-tools" "platforms;android-34" "build-tools;34.0.0"
print("✅ Android SDK 安装完成")

In [ ]:
# 3. 克隆项目并安装依赖
!git clone https://github.com/Michael-YuQ/python-calculator-android.git
%cd python-calculator-android/selfagent
!npm install
print("✅ 依赖安装完成")

In [ ]:
# 4. 生成 Android 项目
import os
os.environ['CI'] = '1'
!npx expo prebuild --platform android --clean --non-interactive
print("✅ Android 项目生成完成")

In [ ]:
# 5. 手动打包 JS Bundle (关键步骤!)
!mkdir -p android/app/src/main/assets

# 先清理可能存在的旧 bundle
!rm -f android/app/src/main/assets/index.android.bundle

# 使用 npx expo export 生成 bundle (更可靠)
!npx expo export --platform android --output-dir dist

# 复制生成的 bundle 到正确位置
!cp dist/bundles/android-*.js android/app/src/main/assets/index.android.bundle 2>/dev/null || \
    cp dist/_expo/static/js/android/*.js android/app/src/main/assets/index.android.bundle 2>/dev/null || \
    echo '尝试使用 react-native bundle...'

# 如果 expo export 失败，使用 react-native bundle 作为备选
import os
bundle_path = 'android/app/src/main/assets/index.android.bundle'
if not os.path.exists(bundle_path) or os.path.getsize(bundle_path) < 1000:
    print('使用 react-native bundle 重新生成...')
    os.system('''npx react-native bundle \
        --platform android \
        --dev false \
        --entry-file index.js \
        --bundle-output android/app/src/main/assets/index.android.bundle \
        --assets-dest android/app/src/main/res \
        --reset-cache \
        --minify true''')

# 验证 bundle 文件
if os.path.exists(bundle_path):
    size = os.path.getsize(bundle_path)
    print(f"✅ JS Bundle 已生成: {size/1024:.1f} KB")
    if size < 10000:
        print("⚠️ 警告: Bundle 文件太小，可能有问题")
    # 显示 bundle 前几行确认内容
    with open(bundle_path, 'r', errors='ignore') as f:
        preview = f.read(200)
        print(f"Bundle 预览: {preview[:100]}...")
else:
    print("❌ Bundle 生成失败")
    exit(1)

In [ ]:
# 6. 修改 build.gradle 确保 bundle 被包含
build_gradle = 'android/app/build.gradle'

with open(build_gradle, 'r') as f:
    content = f.read()

# 确保 assets 目录被包含在构建中
if 'sourceSets' not in content or 'assets.srcDirs' not in content:
    # 在 android { 块内添加 sourceSets 配置
    insert_point = content.find('android {')
    if insert_point > 0:
        # 找到 android { 后的第一个 { 对应的位置
        brace_count = 0
        for i, c in enumerate(content[insert_point:]):
            if c == '{':
                brace_count += 1
                if brace_count == 1:
                    insert_idx = insert_point + i + 1
                    break
        
        sourceSets_config = '''
    sourceSets {
        main {
            assets.srcDirs = ['src/main/assets']
        }
    }
'''
        content = content[:insert_idx] + sourceSets_config + content[insert_idx:]
        with open(build_gradle, 'w') as f:
            f.write(content)
        print("✅ build.gradle 已添加 sourceSets 配置")
else:
    print("✅ sourceSets 配置已存在")

# 列出 assets 目录内容确认
!ls -la android/app/src/main/assets/

In [ ]:
# 7. 构建 Release APK
import os
os.environ['ANDROID_HOME'] = os.path.expanduser('~/android-sdk')
os.environ['ANDROID_SDK_ROOT'] = os.environ['ANDROID_HOME']

# 再次确认 bundle 存在
bundle_check = '../android/app/src/main/assets/index.android.bundle'
if not os.path.exists(bundle_check.replace('../', '')):
    print('⚠️ Bundle 不存在，重新生成...')
    os.chdir('..')
    os.system('mkdir -p android/app/src/main/assets')
    os.system('''npx react-native bundle \
        --platform android \
        --dev false \
        --entry-file index.js \
        --bundle-output android/app/src/main/assets/index.android.bundle \
        --assets-dest android/app/src/main/res''')
    os.chdir('android')

%cd android
!chmod +x gradlew

# 列出 assets 目录确认
print('Assets 目录内容:')
!ls -la app/src/main/assets/

# 清理并构建，跳过 JS bundle 任务因为我们已经手动生成了
!./gradlew clean
!./gradlew assembleRelease -x bundleReleaseJsAndAssets --no-daemon --stacktrace
print("✅ APK 构建完成")

In [ ]:
# 8. 验证并修复 APK 中的 bundle
import os
import subprocess

apk_path = 'app/build/outputs/apk/release/app-release.apk'
bundle_src = 'app/src/main/assets/index.android.bundle'

if os.path.exists(apk_path):
    print("APK 内容检查:")
    result = subprocess.run(['unzip', '-l', apk_path], capture_output=True, text=True)
    
    has_bundle = 'index.android.bundle' in result.stdout
    
    if has_bundle:
        print("✅ Bundle 已正确打包到 APK 中!")
        # 显示 bundle 在 APK 中的信息
        for line in result.stdout.split('\n'):
            if 'index.android.bundle' in line or 'assets' in line:
                print(line)
    else:
        print("❌ APK 中没有找到 bundle，手动添加...")
        
        # 解压 APK
        !rm -rf temp_apk
        !mkdir -p temp_apk
        !unzip -q {apk_path} -d temp_apk
        
        # 添加 bundle
        !mkdir -p temp_apk/assets
        !cp {bundle_src} temp_apk/assets/index.android.bundle
        
        # 重新打包 (不签名的 APK)
        !rm -f app-release-fixed.apk
        os.chdir('temp_apk')
        !zip -r ../app-release-fixed-unsigned.apk .
        os.chdir('..')
        
        # 使用 zipalign 和 apksigner 签名
        # 生成一个临时 keystore
        !keytool -genkey -v -keystore temp.keystore -alias temp -keyalg RSA -keysize 2048 -validity 10000 -storepass android -keypass android -dname "CN=Temp, OU=Temp, O=Temp, L=Temp, ST=Temp, C=US" 2>/dev/null || true
        
        # zipalign
        !~/android-sdk/build-tools/34.0.0/zipalign -v 4 app-release-fixed-unsigned.apk app-release-aligned.apk
        
        # 签名
        !~/android-sdk/build-tools/34.0.0/apksigner sign --ks temp.keystore --ks-pass pass:android --key-pass pass:android --out app-release-fixed.apk app-release-aligned.apk
        
        # 清理
        !rm -rf temp_apk app-release-fixed-unsigned.apk app-release-aligned.apk temp.keystore
        
        if os.path.exists('app-release-fixed.apk'):
            print("✅ 已创建修复后的 APK: app-release-fixed.apk")
            # 验证修复后的 APK
            !unzip -l app-release-fixed.apk | grep -E 'index.android.bundle|assets'
        else:
            print("❌ 修复失败")
else:
    print("❌ APK 未找到")
    !find . -name "*.apk" -type f

In [ ]:
# 9. 下载 APK
from google.colab import files
import shutil, os

# 优先使用修复后的 APK
if os.path.exists('app-release-fixed.apk'):
    files.download('app-release-fixed.apk')
    print("✅ 修复后的 APK 已下载")
elif os.path.exists('app/build/outputs/apk/release/app-release.apk'):
    shutil.copy('app/build/outputs/apk/release/app-release.apk', 'SelfAgent.apk')
    files.download('SelfAgent.apk')
    print("✅ APK 已下载")
else:
    print("❌ 未找到 APK")
    !find .. -name "*.apk" -type f 2>/dev/null